In [1]:
from statsmodels.tsa.stattools import adfuller
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import json

In [33]:
# f = open()
data_df = pd.read_json('src/api/data/AIR_Q_TU_Graz/output_chunk_6_clean.json')

In [34]:
data_df
data_df = data_df.drop(columns=['tvoc'])

In [35]:
data_df


,oxygen,timestamp,health,dewpt,TypPS,gauss,sound,temperature,co2
0,20.764,2023-10-19 07:13:12,761,10.067,1.7,0.209877,72.60000,17.708,617.8
1,20.763,2023-10-19 07:13:16,730,10.078,1.6,0.097164,74.72000,17.716,619.3
2,20.763,2023-10-19 07:13:20,723,10.094,1.6,0.117306,75.02000,17.721,620.8
3,20.763,2023-10-19 07:13:26,730,10.099,1.8,0.396152,75.25000,17.724,622.5
4,20.763,2023-10-19 07:13:30,743,10.096,1.9,0.841833,75.05001,17.726,623.8
...,...,...,...,...,...,...,...,...,...
78396,20.789,2023-10-31 23:59:25,931,11.594,2.1,0.545161,31.24000,18.544,473.6
78397,20.789,2023-10-31 23:59:29,944,11.592,1.8,0.651724,31.36000,18.544,473.8
78398,20.788,2023-10-31 23:59:33,953,11.589,1.7,0.891126,31.47000,18.542,474.1
78399,20.788,2023-10-31 23:59:38,962,11.590,1.6,0.398593,31.47000,18.543,474.6


In [50]:
data_df['timestamp'] = pd.to_datetime(data_df['timestamp'])


In [55]:
min(data_df['timestamp'])

Timestamp('2023-10-19 07:13:12')

In [56]:
max(data_df['timestamp'])

Timestamp('2023-10-31 23:59:43')

In [54]:
#my_range = pd.date_range(start=data_df['Day'].min(), end=data_df['Day'].max(), freq='D')
data_df.index = data_df["timestamp"]
data_df = data_df.sort_index(ascending=True)

In [57]:
my_range = pd.date_range(start=data_df['timestamp'].min(), end=data_df['timestamp'].max(), freq='4S')
my_range

/var/folders/xk/ct1qcm4j5ljdj441cx4jphvjp9l86x/T/ipykernel_24106/3396330028.py:1: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  my_range = pd.date_range(start=data_df['timestamp'].min(), end=data_df['timestamp'].max(), freq='4S')


DatetimeIndex(['2023-10-19 07:13:12', '2023-10-19 07:13:16',
               '2023-10-19 07:13:20', '2023-10-19 07:13:24',
               '2023-10-19 07:13:28', '2023-10-19 07:13:32',
               '2023-10-19 07:13:36', '2023-10-19 07:13:40',
               '2023-10-19 07:13:44', '2023-10-19 07:13:48',
               ...
               '2023-10-31 23:59:04', '2023-10-31 23:59:08',
               '2023-10-31 23:59:12', '2023-10-31 23:59:16',
               '2023-10-31 23:59:20', '2023-10-31 23:59:24',
               '2023-10-31 23:59:28', '2023-10-31 23:59:32',
               '2023-10-31 23:59:36', '2023-10-31 23:59:40'],
              dtype='datetime64[ns]', length=274298, freq='4s')

In [61]:
len(my_range[~my_range.isin(data_df.index)])


254600

In [64]:
data_df.size

705609

In [60]:
my_range[~my_range.isin(data_df.index)]

DatetimeIndex(['2023-10-19 07:13:24', '2023-10-19 07:13:28',
               '2023-10-19 07:13:32', '2023-10-19 07:13:40',
               '2023-10-19 07:13:44', '2023-10-19 07:13:52',
               '2023-10-19 07:13:56', '2023-10-19 07:14:04',
               '2023-10-19 07:14:08', '2023-10-19 07:14:12',
               ...
               '2023-10-31 23:59:00', '2023-10-31 23:59:04',
               '2023-10-31 23:59:08', '2023-10-31 23:59:12',
               '2023-10-31 23:59:20', '2023-10-31 23:59:24',
               '2023-10-31 23:59:28', '2023-10-31 23:59:32',
               '2023-10-31 23:59:36', '2023-10-31 23:59:40'],
              dtype='datetime64[ns]', length=254600, freq=None)

In [39]:
data_df.size

705609

In [66]:
data_df.duplicated().value_counts()
data_df = data_df.drop_duplicates()



In [67]:
data_df.size

705609

In [68]:
data_df_1m = data_df.groupby(pd.Grouper(key='timestamp', freq='1min')).mean()

In [69]:
data_df_1m

,oxygen,health,dewpt,TypPS,gauss,sound,temperature,co2
timestamp,,,,,,,,
2023-10-19 07:13:00,20.764444,737.111111,10.087778,1.722222,0.375576,74.277780,17.724889,623.055544
2023-10-19 07:14:00,20.769000,738.500000,10.103000,1.450000,0.701922,74.280000,17.746500,625.550000
2023-10-19 07:15:00,20.763000,666.333333,10.164000,1.833333,0.610247,74.803333,17.807333,632.800000
2023-10-19 07:16:00,20.765556,717.444444,10.180111,1.611111,0.340111,73.571112,17.827889,644.177778
2023-10-19 07:17:00,20.771500,761.666667,10.200333,1.550000,0.410790,72.431667,17.863000,655.483333
...,...,...,...,...,...,...,...,...
2023-10-31 23:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-31 23:56:00,20.787600,944.400000,11.591000,2.020000,0.476600,31.594000,18.544600,474.820000
2023-10-31 23:57:00,20.787000,974.666667,11.586000,1.416667,0.413754,31.476667,18.545167,474.316667


In [72]:
df_filled = data_df_1m.bfill()

In [73]:
df_filled

,oxygen,health,dewpt,TypPS,gauss,sound,temperature,co2
timestamp,,,,,,,,
2023-10-19 07:13:00,20.764444,737.111111,10.087778,1.722222,0.375576,74.277780,17.724889,623.055544
2023-10-19 07:14:00,20.769000,738.500000,10.103000,1.450000,0.701922,74.280000,17.746500,625.550000
2023-10-19 07:15:00,20.763000,666.333333,10.164000,1.833333,0.610247,74.803333,17.807333,632.800000
2023-10-19 07:16:00,20.765556,717.444444,10.180111,1.611111,0.340111,73.571112,17.827889,644.177778
2023-10-19 07:17:00,20.771500,761.666667,10.200333,1.550000,0.410790,72.431667,17.863000,655.483333
...,...,...,...,...,...,...,...,...
2023-10-31 23:55:00,20.787600,944.400000,11.591000,2.020000,0.476600,31.594000,18.544600,474.820000
2023-10-31 23:56:00,20.787600,944.400000,11.591000,2.020000,0.476600,31.594000,18.544600,474.820000
2023-10-31 23:57:00,20.787000,974.666667,11.586000,1.416667,0.413754,31.476667,18.545167,474.316667


In [75]:
my_range_in1m = pd.date_range(start=df_filled.index.min(), end=df_filled.index.max(), freq='1min')


In [77]:
len(my_range_in1m[~my_range_in1m.isin(df_filled.index)])


0

In [79]:
df_filled["timestamp"] = df_filled.index

In [80]:
df_filled.to_json('./src/api/data/AIR_Q_TU_Graz/output_chunk_6_cut_out_coarse_grained_1minute.json', orient="records")

In [12]:
grouped = data_df.groupby('Entity')

In [13]:
data_df['Entity'].unique()

array(['Afghanistan', 'Africa', 'Albania', 'Algeria', 'American Samoa',
       'Andorra', 'Angola', 'Anguilla', 'Antigua and Barbuda',
       'Argentina', 'Armenia', 'Aruba', 'Asia', 'Asia excl. China',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia', 'Bonaire Sint Eustatius and Saba',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Cayman Islands', 'Central African Republic', 'Chad', 'Chile',
       'China', 'Colombia', 'Comoros', 'Congo', 'Cook Islands',
       'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Curacao',
       'Cyprus', 'Czechia', 'Democratic Republic of Congo', 'Denmark',
       'Djibouti', 'Dominica', 'Dominican Republic', 'East Timor',
       'Ecuador', 'Egypt', 'El Salvador', 'Equator

In [14]:
data_df = data_df[data_df["Entity"] == 'Austria']

In [15]:
data_df

,Entity,Code,Day,Weekly cases
24930,Austria,AUT,2020-01-10,0
24931,Austria,AUT,2020-01-11,0
24932,Austria,AUT,2020-01-12,0
24933,Austria,AUT,2020-01-13,0
24934,Austria,AUT,2020-01-14,0
...,...,...,...,...
26587,Austria,AUT,2024-07-24,12
26588,Austria,AUT,2024-07-25,12
26589,Austria,AUT,2024-07-26,12
26590,Austria,AUT,2024-07-27,12


In [16]:
data_df.isna().sum()

Entity          0
Code            0
Day             0
Weekly cases    0
dtype: int64

In [17]:
data_df.index = pd.DatetimeIndex(pd.to_datetime(data_df["Day"])) #.sort_index(ascending=True, inplace=False)


In [18]:
data_df

,Entity,Code,Day,Weekly cases
Day,,,,
2020-01-10,Austria,AUT,2020-01-10,0
2020-01-11,Austria,AUT,2020-01-11,0
2020-01-12,Austria,AUT,2020-01-12,0
2020-01-13,Austria,AUT,2020-01-13,0
2020-01-14,Austria,AUT,2020-01-14,0
...,...,...,...,...
2024-07-24,Austria,AUT,2024-07-24,12
2024-07-25,Austria,AUT,2024-07-25,12
2024-07-26,Austria,AUT,2024-07-26,12


In [16]:
my_range = pd.date_range(start=data_df['Day'].min(), end=data_df['Day'].max(), freq='D')


In [17]:
len(my_range[~my_range.isin(data_df.index)])


0

In [18]:
my_range.drop_duplicates()

DatetimeIndex(['2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04',
               '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08',
               '2020-03-09', '2020-03-10',
               ...
               '2024-05-03', '2024-05-04', '2024-05-05', '2024-05-06',
               '2024-05-07', '2024-05-08', '2024-05-09', '2024-05-10',
               '2024-05-11', '2024-05-12'],
              dtype='datetime64[ns]', length=1534, freq='D')

In [19]:
print(data_df.shape)
new_df = pd.DataFrame()
new_df.index = my_range
new_df["Day"] = my_range
data_df = pd.merge(data_df, new_df, how="outer", right_index=True, left_index=True, validate="many_to_many")

(1534, 5)


In [20]:
data_df.index.drop_duplicates()
print(data_df)


             Entity Code       Day_x  Total confirmed deaths due to COVID-19  \
2020-03-01  Austria  AUT  2020-03-01                                     NaN   
2020-03-02  Austria  AUT  2020-03-02                                     NaN   
2020-03-03  Austria  AUT  2020-03-03                                     NaN   
2020-03-04  Austria  AUT  2020-03-04                                     NaN   
2020-03-05  Austria  AUT  2020-03-05                                     NaN   
...             ...  ...         ...                                     ...   
2024-05-08  Austria  AUT  2024-05-08                                 22534.0   
2024-05-09  Austria  AUT  2024-05-09                                 22534.0   
2024-05-10  Austria  AUT  2024-05-10                                 22534.0   
2024-05-11  Austria  AUT  2024-05-11                                 22534.0   
2024-05-12  Austria  AUT  2024-05-12                                 22534.0   

            Total confirmed cases of CO

In [21]:
data_df = data_df.groupby(data_df.index).last()

In [22]:
data_df = data_df.drop_duplicates(keep='last')

In [23]:
print(f"Duplicates: {data_df.index.duplicated()}")


Duplicates: [False False False ... False False False]


In [24]:
data_df = data_df.drop(columns=['Day_x', 'Day_y'])


In [25]:
data_df["Day"] = data_df.index

In [26]:
data_df.isna().sum()

Entity                                     0
Code                                       0
Total confirmed deaths due to COVID-19    14
Total confirmed cases of COVID-19          0
Day                                        0
dtype: int64

In [27]:
data_df = data_df.fillna(0)

In [19]:
data_df

,Entity,Code,Day,Weekly cases
Day,,,,
2020-01-10,Austria,AUT,2020-01-10,0
2020-01-11,Austria,AUT,2020-01-11,0
2020-01-12,Austria,AUT,2020-01-12,0
2020-01-13,Austria,AUT,2020-01-13,0
2020-01-14,Austria,AUT,2020-01-14,0
...,...,...,...,...
2024-07-24,Austria,AUT,2024-07-24,12
2024-07-25,Austria,AUT,2024-07-25,12
2024-07-26,Austria,AUT,2024-07-26,12


In [20]:
data_df1 = pd.read_csv('src/api/data/weekly-covid-deaths.csv')

In [21]:
grouped = data_df1.groupby('Entity')


In [22]:
data_df1 = data_df1[data_df1["Entity"] == 'Austria']

In [23]:
data_df1.isna().sum()

Entity           0
Code             0
Day              0
Weekly deaths    0
dtype: int64

In [24]:
data_df1.index = pd.DatetimeIndex(pd.to_datetime(data_df1["Day"])) #.sort_index(ascending=True, inplace=False)


In [32]:
data_df = data_df.drop(columns=['Entity', 'Code'])
data_df

,Weekly cases
Day,
2020-01-10,0
2020-01-11,0
2020-01-12,0
2020-01-13,0
2020-01-14,0
...,...
2024-07-24,12
2024-07-25,12
2024-07-26,12


In [33]:
data_df1 = data_df1.drop(columns=['Entity', 'Code'])
data_df1

,Weekly deaths
Day,
2020-01-10,0
2020-01-11,0
2020-01-12,0
2020-01-13,0
2020-01-14,0
...,...
2024-07-24,0
2024-07-25,0
2024-07-26,0


In [34]:
joint = data_df.join(data_df1, how='inner', on="Day")

In [36]:
joint

,Weekly cases,Weekly deaths
Day,,
2020-01-10,0,0
2020-01-11,0,0
2020-01-12,0,0
2020-01-13,0,0
2020-01-14,0,0
...,...,...
2024-07-24,12,0
2024-07-25,12,0
2024-07-26,12,0


In [38]:
joint["timestamp"] = joint.index

In [39]:
joint.to_json('./src/api/data/non-cumulative-deaths-and-cases-covid_14-08-24_Austria.json', orient="records")